In [1]:
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation,Lambda,Bidirectional
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform
import tensorflow as tf
import re

Using TensorFlow backend.


In [2]:
import tensorflow as tf
import tensorflow_hub as hub
module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/3"
# Import the Universal Sentence Encoder's TF Hub module
embed = hub.Module(module_url)

# Compute a representation for each message, showing various lengths supported.
messages = ["That band rocks!", "That song is really cool."]

with tf.Session() as session:
  session.run([tf.global_variables_initializer(), tf.tables_initializer()])
  message_embeddings = session.run(embed(messages))

W0606 04:35:23.557449 140527414105984 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


Instructions for updating:
Colocations handled automatically by placer.


W0606 04:35:35.525181 140527414105984 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0606 04:35:39.465465 140527414105984 saver.py:1483] Saver not created because there are no variables in the graph to restore


In [3]:
def get_dataframe(filename):
    lines = open(filename, 'r').read().splitlines()
    data = []
    for i in range(0, len(lines)):
        label = lines[i].split(',')[1]
        tweet= lines[i].split(',')[0]
        tweet= re.sub('[^A-Za-z0-9 ,\?\'\"-._\+\!/\`@=;:]+', '', tweet)
        data.append([tweet,label])

    df = pd.DataFrame(data, columns=['tweet', 'label'])
    df.label = df.label.astype('category')
    return df

df_train = get_dataframe('/content/Processed+Data-Train.csv')
df_train.head()

,tweet,label
0,ebola symptoms early treatment means a much be...,0
1,thinking about telling leadership to make indi...,1
2,this is nurse 2 is at this very moment being r...,0
3,fourth doctor dies of ebola in sierra leone af...,2
4,ebola is not a death sentence early treatment ...,0


In [0]:
train_text = df_train['tweet'].tolist()
train_text = np.array(train_text, dtype=object)[:, np.newaxis] #each tweet becomes a new list in the list

train_label = np.asarray(pd.get_dummies(df_train.label), dtype = np.int8)# 1 hot vectors



In [0]:
def UniversalEmbedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)), 
    	signature="default", as_dict=True)["default"]


In [6]:
input_text = Input(shape=(1,), dtype=tf.string)
embedding = Lambda(UniversalEmbedding,output_shape=(512,))(input_text)
dense = Dense(256, activation='relu')(embedding)
dense = Dense(128, activation='relu')(embedding)
pred = Dense(7, activation='softmax')(dense)
model = Model(inputs=[input_text], outputs=pred)
model.compile(loss='categorical_crossentropy', 
	optimizer='adam', metrics=['accuracy'])


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0606 04:35:56.631726 140527414105984 saver.py:1483] Saver not created because there are no variables in the graph to restore


In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1)                 0         
_________________________________________________________________
lambda_1 (Lambda)            (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               65664     
_________________________________________________________________
dense_3 (Dense)              (None, 7)                 903       
Total params: 66,567
Trainable params: 66,567
Non-trainable params: 0
_________________________________________________________________


In [9]:
def get_dataframe(filename):
    lines = open(filename, 'r').read().splitlines()
    data = []
    for i in range(0, len(lines)):
        label = lines[i].split(',')[1]
        tweet= lines[i].split(',')[0]
        tweet= re.sub('[^A-Za-z0-9 ,\?\'\"-._\+\!/\`@=;:]+', '', tweet)
        data.append([tweet,label])

    df = pd.DataFrame(data, columns=['tweet', 'label'])
    df.label = df.label.astype('category')
    return df

df_test = get_dataframe('/content/Processed+Data-Test.csv')
df_test.head()

,tweet,label
0,the us centre for disease control cdc issued t...,6
1,breaking employment agency in berlin placed in...,0
2,abuja schools resume screen pupils for symptom...,3
3,people are now apparently faking ebola to get ...,0
4,360nobs ebola disease not death sentence says ...,6


In [0]:
test_text = df_test['tweet'].tolist()
test_text = np.array(test_text, dtype=object)[:, np.newaxis] #each tweet becomes a new list in the list

test_label = np.asarray(pd.get_dummies(df_test.label), dtype = np.int8)# 1 hot vectors


In [11]:
from keras import backend as K
with tf.Session() as session:
  K.set_session(session)
  session.run(tf.global_variables_initializer())
  session.run(tf.tables_initializer())
  model.fit(train_text, 
            train_label,
            epochs=80,
            batch_size=16)
  loss, acc = model.evaluate(test_text,test_label)
print()
print("Test accuracy = ", acc)

Instructions for updating:
Use tf.cast instead.


W0606 04:39:44.440982 140527414105984 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_ops.py:3066: to_int32 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.cast instead.


Epoch 1/80
1583/1583 [==============================] - 32s 20ms/step - loss: 1.6984 - acc: 0.3860
Epoch 2/80
1583/1583 [==============================] - 29s 18ms/step - loss: 1.3418 - acc: 0.5351
Epoch 3/80
1583/1583 [==============================] - 29s 18ms/step - loss: 1.1324 - acc: 0.6096
Epoch 4/80
1583/1583 [==============================] - 28s 18ms/step - loss: 1.0107 - acc: 0.6450
Epoch 5/80
1583/1583 [==============================] - 28s 18ms/step - loss: 0.9254 - acc: 0.6810
Epoch 6/80
1583/1583 [==============================] - 28s 18ms/step - loss: 0.8778 - acc: 0.7081
Epoch 7/80
1583/1583 [==============================] - 28s 18ms/step - loss: 0.8286 - acc: 0.7138
Epoch 8/80
1583/1583 [==============================] - 28s 18ms/step - loss: 0.8059 - acc: 0.7214
Epoch 9/80
1583/1583 [==============================] - 28s 18ms/step - loss: 0.7750 - acc: 0.7353
Epoch 10/80
1583/1583 [==============================] - 29s 18ms/step - loss: 0.7514 - acc: 0.7429
Epoch 11/